In [1]:
%dependency /add com.microsoft.semantic-kernel:semantickernel-api:1.2.0
%dependency /add com.microsoft.semantic-kernel:semantickernel-bom:1.2.0
%dependency /add com.microsoft.semantic-kernel:semantickernel-parent:1.2.0
%dependency /add com.microsoft.semantic-kernel:semantickernel-aiservices-openai:1.2.0
%dependency /add org.slf4j:slf4j-jdk14:2.0.12


%dependency /override io.netty:netty-codec-http:4.1.107.Final
%dependency /override io.netty:netty-common:4.1.107.Final
%dependency /override io.netty:netty-buffer:4.1.107.Final
%dependency /override io.netty:netty-transport:4.1.107.Final
%dependency /override io.netty:netty-resolver:4.1.107.Final
%dependency /override io.netty:netty-codec:4.1.107.Final
%dependency /override io.netty:netty-handler:4.1.107.Final
%dependency /override io.netty:netty-transport-native-unix-common:4.1.107.Final
%dependency /override io.netty:netty-codec-http2:4.1.107.Final
%dependency /override io.netty:netty-transport-native-epoll:4.1.107.Final
%dependency /override io.netty:netty-handler-proxy:4.1.107.Final
%dependency /override io.projectreactor:reactor-core:3.4.36

%dependency /resolve

Adding dependency com.microsoft.semantic-kernel:semantickernel-api:1.2.0
Adding dependency com.microsoft.semantic-kernel:semantickernel-bom:1.2.0
Adding dependency com.microsoft.semantic-kernel:semantickernel-parent:1.2.0
Adding dependency com.microsoft.semantic-kernel:semantickernel-aiservices-openai:1.2.0
Adding dependency org.slf4j:slf4j-jdk14:2.0.12
Adding dependency override io.netty:netty-codec-http:4.1.107.Final
Adding dependency override io.netty:netty-common:4.1.107.Final
Adding dependency override io.netty:netty-buffer:4.1.107.Final
Adding dependency override io.netty:netty-transport:4.1.107.Final
Adding dependency override io.netty:netty-resolver:4.1.107.Final
Adding dependency override io.netty:netty-codec:4.1.107.Final
Adding dependency override io.netty:netty-handler:4.1.107.Final
Adding dependency override io.netty:netty-transport-native-unix-common:4.1.107.Final
Adding dependency override io.netty:netty-codec-http2:4.1.107.Final
Adding dependency override io.netty:netty

In [2]:
import java.nio.file.Path;

In [3]:
import com.azure.ai.openai.OpenAIAsyncClient;
import com.azure.ai.openai.OpenAIClientBuilder;
import com.azure.core.credential.AzureKeyCredential;
import com.azure.core.credential.KeyCredential;

In [4]:
import com.microsoft.semantickernel.Kernel;
import com.microsoft.semantickernel.Kernel.Builder;
import com.microsoft.semantickernel.exceptions.ConfigurationException;
import com.microsoft.semantickernel.orchestration.FunctionResult;
import com.microsoft.semantickernel.orchestration.PromptExecutionSettings;
import com.microsoft.semantickernel.plugin.KernelPluginFactory;
import com.microsoft.semantickernel.semanticfunctions.KernelFunction;
import com.microsoft.semantickernel.semanticfunctions.KernelFunctionArguments;
import com.microsoft.semantickernel.semanticfunctions.KernelFunctionFromPrompt;
import com.microsoft.semantickernel.services.textcompletion.TextGenerationService;

import com.microsoft.semantickernel.aiservices.openai.chatcompletion.OpenAIChatCompletion;
import com.microsoft.semantickernel.services.chatcompletion.ChatCompletionService;

In [5]:
String AZURE_KEY = "Your Azure OpenAI Service API Key";
String AZURE_ENDPOINT = "Your Azure OpenAI Service Endpoint";

In [6]:
OpenAIAsyncClient client = new OpenAIClientBuilder()
        .credential(new AzureKeyCredential(AZURE_KEY))
        .endpoint(AZURE_ENDPOINT)
        .buildAsyncClient();

In [7]:
ChatCompletionService chatGenerationService = OpenAIChatCompletion.builder()
                        .withOpenAIAsyncClient(client)
                        .withModelId("Your Azure OpenAI Service gpt-3.5-turbo-16k deployment name")
                        .build();

In [8]:

var codePlugin = KernelFunctionFromPrompt.builder()
                        .withTemplate("System: You are a java developer 。 User:{{$input}}")
                        .withDefaultExecutionSettings(
                            PromptExecutionSettings.builder()
                                .withMaxTokens(2000)
                                .withTemperature(0.7)
                                .build())
                        .build();

In [9]:
Kernel kernel = Kernel.builder()
    .withAIService(ChatCompletionService.class, chatGenerationService)
    .build();

In [10]:
var result = kernel
            .invokeAsync(codePlugin)
            .withArguments(
                KernelFunctionArguments.builder()
                    .withInput("Generate a bubble algorithm method with Java")
                    .build())
            .block();

In [11]:
result.getResult();

Sure! Here's an example of a bubble sort algorithm implemented in Java:

```java
public class BubbleSort {
    public static void bubbleSort(int[] arr) {
        int n = arr.length;
        for (int i = 0; i < n-1; i++) {
            for (int j = 0; j < n-i-1; j++) {
                if (arr[j] > arr[j+1]) {
                    // Swap arr[j] and arr[j+1]
                    int temp = arr[j];
                    arr[j] = arr[j+1];
                    arr[j+1] = temp;
                }
            }
        }
    }

    public static void main(String[] args) {
        int[] arr = {64, 34, 25, 12, 22, 11, 90};
        bubbleSort(arr);
        System.out.println("Sorted array:");
        for (int i = 0; i < arr.length; i++) {
            System.out.print(arr[i] + " ");
        }
    }
}
```

This code defines a `BubbleSort` class with a `bubbleSort` method that takes an array of integers as input and sorts it using the bubble sort algorithm. The `main` method demonstrates how to use this 

In [12]:
var translatePlugin =  KernelPluginFactory
                    .importPluginFromDirectory(
                        Path.of("../../../", "plugins"),
                        "TranslatePlugin",
                        null);

In [13]:
var result = kernel
            .invokeAsync(translatePlugin.get("MultiLanguage"))
            .withArguments(
                KernelFunctionArguments.builder()
                    .withVariable("input", "hello world")
                    .withVariable("language", "fr")
                    .build())
            .block();

In [14]:
result.getResult();

Bonjour le monde

In [15]:
import com.microsoft.semantickernel.semanticfunctions.annotations.DefineKernelFunction;
import com.microsoft.semantickernel.semanticfunctions.annotations.KernelFunctionParameter;

In [16]:
public static class CustomPlugin {

    @DefineKernelFunction(description = "Get news from the web", name = "NewsFunction")
    public static String askNews(
        @KernelFunctionParameter(description = "getlocation", name = "location") String location) {
        return "Get news from the " + location + ".";
    }

    @DefineKernelFunction(description = "Search Weather in a city", name = "WeatherFunction")
    public String askWeather(
        @KernelFunctionParameter(description = "get city", name = "city") String city) {
        return city + "’s weather is 30 celsius degree , and very hot.";
    }

    @DefineKernelFunction(description = "Search docs", name = "DocsFunction")
    public String askDocs(
        @KernelFunctionParameter(description = "get docs", name = "docs") String docs) {
        return "ask docs :" + docs;
    }
}

In [17]:
Builder kernelBuilder = Kernel.builder()
            .withAIService(ChatCompletionService.class, chatGenerationService);

In [18]:

var customPlugin =KernelPluginFactory.createFromObject(new CustomPlugin(),"CustomPlugin")

In [19]:
kernelBuilder.withPlugin(customPlugin);

com.microsoft.semantickernel.Kernel$Builder@1f50274e

In [20]:
Kernel kernel = kernelBuilder.build();

In [21]:

var result = kernel.invokeAsync(kernel.getFunction("CustomPlugin", "NewsFunction"))
                        .withArguments(
                            KernelFunctionArguments.builder()
                                .withVariable("location", "guangzhou")
                                .build())
                        .block();

In [22]:
result.getResult();

Get news from the guangzhou.